## Load data

In [1]:
import theano
theano.config.mode = 'NanGuardMode'#theano.compile. (nan_is_error=True, inf_is_error=False, big_is_error=False)
theano.config.NanGuardMode.nan_is_error = True
theano.config.NanGuardMode.inf_is_error = False
theano.config.NanGuardMode.big_is_error = False
import numpy as np
from keras.utils.np_utils import to_categorical
import keras
import os
#os.putenv('PYTHONHASHSEED', '0')

Using Theano backend.


In [2]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [3]:
datapath = '/media/sf_VBox_Shared/timeseries/PAMAP2_Dataset/slidingwindow512cleaned/'

In [4]:
ext = '.npy'
X_train = np.load(datapath+'X_train'+ext)
y_train_binary = np.load(datapath+'y_train_binary'+ext)
X_val = np.load(datapath+'X_val'+ext)
y_val_binary = np.load(datapath+'y_val_binary'+ext)
X_test = np.load(datapath+'X_test'+ext)
y_test_binary = np.load(datapath+'y_test_binary'+ext)

In [5]:
num_classes = y_train_binary.shape[1]
print(num_classes)

12


In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, Flatten, MaxPooling1D
from keras.optimizers import Adam
import keras.backend as K

In [7]:
import sys
import os
import numpy as np
sys.path.insert(0, os.path.abspath('..'))

from mcfly import modelgen, find_architecture

## Replicate the NaN model

In [8]:
params = {'filters': np.array([98, 67]), 'fc_hidden_nodes': 62, 'regularization_rate': 0.03708604649827486, 'learning_rate': 0.00019850152863738226}

We make a copy of the model, to start training from fresh.

In [9]:
# Check some theano settings

print(K.floatx())
print(K.epsilon())
#K.set_epsilon(1e-5)
#K.set_floatx = 'float32'
print(theano.config.optimizer)
#theano.config.optimizer = 'fast_run'
print(theano.config.nvcc.fastmath)
print(theano.config.mode)

float32
1e-07
fast_run
False
NanGuardMode


In [10]:
theano.config.NanGuardMode.inf_is_error

False

We're going to train the model on the complete data set, and save the model after each epoch

In [11]:
np.random.seed(123)
model = modelgen.generate_CNN_model(X_train.shape, num_classes, 
                                              params['filters'], 
                                              params['fc_hidden_nodes'],
                                              params['learning_rate'], 
                                              params['regularization_rate'])

nr_epochs = 1
datasize = X_train.shape[0]#1000
history = model.fit(X_train[:datasize,:,:], y_train_binary[:datasize,:],
              nb_epoch=nr_epochs, validation_data=(X_val, y_val_binary), 
                              callbacks=[keras.callbacks.ModelCheckpoint(datapath+'weights2.{epoch:02d}-{val_loss:.2f}.hdf5')])

Train on 12497 samples, validate on 2007 samples
Epoch 1/1


AssertionError: Inf detected
Big value detected
NanGuardMode found an error in an input of this node.
Node:
Elemwise{Composite{((i0 * sqrt(clip((i1 - (i2 ** i3)), i4, i5))) / (i1 - (i6 ** i3)))}}(<TensorType(float32, scalar)>, TensorConstant{1.0}, <TensorType(float32, scalar)>, Elemwise{Add}[(0, 1)].0, TensorConstant{0.0}, TensorConstant{inf}, <TensorType(float32, scalar)>)
The input variable that cause problem:
Elemwise{Composite{((i0 * sqrt(clip((i1 - (i2 ** i3)), i4, i5))) / (i1 - (i6 ** i3)))}} [id A] ''   
 |<TensorType(float32, scalar)> [id B]
 |TensorConstant{1.0} [id C]
 |<TensorType(float32, scalar)> [id D]
 |Elemwise{Add}[(0, 1)] [id E] ''   
 | |TensorConstant{1.0} [id C]
 | |<TensorType(float32, scalar)> [id F]
 |TensorConstant{0.0} [id G]
 |TensorConstant{inf} [id H]
 |<TensorType(float32, scalar)> [id I]


Apply node that caused the error: Elemwise{Composite{((i0 * sqrt(clip((i1 - (i2 ** i3)), i4, i5))) / (i1 - (i6 ** i3)))}}(<TensorType(float32, scalar)>, TensorConstant{

If we plot the training process, we see that the model is probably already overfitting in the first epoch. 

In [ ]:
find_architecture.plotTrainingProcess(history)

In [ ]:
#Load model from the  just before we get the NaNs
#model_copy = keras.models.copy.copy(best_model_copy)
#model_copy.load_weights(datapath+'weights.{epoch:02d}-{val_loss:.2f}.hdf5'.format(epoch=3, val_loss=0.22))
#model_copy.compile(best_model_copy.optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.evaluate(X_train, y_train_binary)

## Inspect model predictions

In [ ]:
datasize = X_val.shape[0]
probs = model.predict_proba(X_val[:datasize,:,:])
probs

In [ ]:
len(model.layers)

In [ ]:
model.layers[0].get_weights()

In [ ]:
# with a Sequential model
get_dens_layer_output = K.function([model.layers[0].input, K.learning_phase()],
                                  [model.layers[0].output])
layer_output = get_dens_layer_output([X_val, 0])[0]
layer_output

In [ ]:
score_val